In [1]:
import django
django.setup()
from coinsapp.models import Coin,Value, Coinproperties
from datetime import datetime, timedelta
from django.utils import timezone
import ccxt

from bokeh.plotting import figure, output_file, show

In [16]:
t = datetime.now()-timedelta(hours=1)
print(datetime.now())
print(t)

2018-01-26 15:28:17.516582
2018-01-26 14:28:17.516451


##### from coinsapp.tasks import get_all_markets

In [2]:
from coinsapp.tasks import get_all_markets
get_all_markets()


('GET', 'https://api.binance.com/api/v1/exchangeInfo', '\nRequest:', {'Accept-Encoding': 'gzip, deflate'}, None)
('GET', 'https://api.binance.com/api/v1/exchangeInfo', '\nResponse:', "{'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'Server': 'nginx', 'Connection': 'keep-alive', 'Date': 'Mon, 05 Feb 2018 14:35:53 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Type': 'application/json'}", u'{"timezone":"UTC","serverTime":1517841353102,"rateLimits":[{"rateLimitType":"REQUESTS","interval":"MINUTE","limit":1200},{"rateLimitType":"ORDERS","interval":"SECOND","limit":10},{"rateLimitType":"ORDERS","interval":"DAY","limit":100000}],"exchangeFilters":[],"symbols":[{"symbol":"ETHBTC","status":"TRADING","baseAsset":"ETH","baseAssetPrecision":8,"quoteAsset":"BTC","quotePrecision":8,"orderTypes":["LIMIT","LIMIT_MAKER","MARKET","STOP_LOSS_LIMIT","TAKE_PROFIT_LIMIT"],"icebergAllowed":true,"filters":[{"filterType":"PRICE_

ImportError: cannot import name get_all_markets

In [3]:
from coinsapp.tasks import get_coin_data
get_coin_data()

('GET', 'https://api.binance.com/api/v1/ticker/24hr', '\nRequest:', {'Accept-Encoding': 'gzip, deflate'}, None)
('GET', 'https://api.binance.com/api/v1/ticker/24hr', '\nResponse:', "{'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'Server': 'nginx', 'Connection': 'keep-alive', 'Date': 'Mon, 05 Feb 2018 14:38:40 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Type': 'application/json'}", u'[{"symbol":"ETHBTC","priceChange":"-0.00222100","priceChangePercent":"-2.180","weightedAvgPrice":"0.10179913","prevClosePrice":"0.10185800","lastPrice":"0.09963700","lastQty":"0.40000000","bidPrice":"0.09963700","bidQty":"4.51000000","askPrice":"0.09973800","askQty":"2.59100000","openPrice":"0.10185800","highPrice":"0.10367200","lowPrice":"0.09875300","volume":"124422.98500000","quoteVolume":"12666.15210385","openTime":1517755120976,"closeTime":1517841520976,"firstId":30041474,"lastId":30450814,"count":409341},{"symbol":"L

DoesNotExist: Coin matching query does not exist.

In [5]:
exchange = ccxt.binance  ({ 'verbose': True })
exchange.load_markets()
list=exchange.symbols
btc_list=filter(lambda k: '/BTC' in k, list)
for market in btc_list:
        c=Coin.objects.update_or_create(coin_name=market)

('GET', 'https://api.binance.com/api/v1/exchangeInfo', '\nRequest:', {'Accept-Encoding': 'gzip, deflate'}, None)
('GET', 'https://api.binance.com/api/v1/exchangeInfo', '\nResponse:', "{'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'Server': 'nginx', 'Connection': 'keep-alive', 'Date': 'Sun, 14 Jan 2018 17:32:14 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Type': 'application/json'}", u'{"timezone":"UTC","serverTime":1515951134299,"rateLimits":[{"rateLimitType":"REQUESTS","interval":"MINUTE","limit":1200},{"rateLimitType":"ORDERS","interval":"SECOND","limit":10},{"rateLimitType":"ORDERS","interval":"DAY","limit":100000}],"exchangeFilters":[],"symbols":[{"symbol":"ETHBTC","status":"TRADING","baseAsset":"ETH","baseAssetPrecision":8,"quoteAsset":"BTC","quotePrecision":8,"orderTypes":["LIMIT","LIMIT_MAKER","MARKET","STOP_LOSS_LIMIT","TAKE_PROFIT_LIMIT"],"icebergAllowed":true,"filters":[{"filterType":"PRICE_

In [52]:
c = Coin.objects.get(coin_name='ADA/BTC')
x=[]
y=[]
for values in c.value_set.all():
    x.append(values.reqtime)
    y.append(values.coin_value)



In [86]:
import numpy as np
from numpy import convolve
 
def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = list(np.convolve(values, weights, 'valid'))
    
    n=0
    
    while (n < window-1):
        n=n+1
        nan = float('nan')
        sma.insert(0,nan)
    return sma

sma = movingaverage (y, 5)

print(sma)

[nan, nan, nan, nan, 5.5820000000000004e-05, 5.575000000000001e-05, 5.576600000000001e-05, 5.589600000000001e-05, 5.606600000000001e-05, 5.6216000000000005e-05, 5.635200000000001e-05]


In [87]:


# output to static HTML file
output_file("lines.html")

# create a new plot with a title and axis labels
p = figure(title="ADA/BTC", x_axis_label='Time', y_axis_label='Price', x_axis_type='datetime', height=250)
p.toolbar.logo = None
# add a line renderer with legend and line thickness
p.line( x,sma, legend="SMA", line_width=2 )
p.circle(x, y, fill_color="red", size=10)
p.line(x, y, line_color="red", legend="ADA/BTC", line_width=2)
p.sizing_mode = 'scale_width'
# show the results
show(p)

In [121]:
from datetime import datetime, timedelta


z=Value.objects.filter(reqtime__lt=datetime.today()-timedelta(minutes=160))
z

<QuerySet []>

In [8]:
from coinsapp.tasks import  remove_old_values

remove_old_values()

In [22]:
    
    t=datetime.now()-timedelta(minutes=10)
    c=Coin.objects.get(coin_name=ticker)
    v=c.value_set.filter(reqtime__lt=t)
    value=(v.value_set.all()['coin_basevolume'].first()-v.value_set.all()['coin_basevolume'].first())/
    v.value_set.all()['coin_basevolume'].first()*100

AttributeError: 'QuerySet' object has no attribute 'coin_set'

In [5]:
from time import strftime
from coinsapp.models import Coin,Value, Coinproperties

c=Coin.objects.get(id=2)
t = datetime.now()-timedelta(minutes=5)
v=c.value_set.filter(reqtime__lt=t).order_by('reqtime')

print('now: ', datetime.now().strftime("%d %H:%M"))
print('gr than: ',t.strftime("%d %H:%M"))

for  key in v:
    print(key.id, " ", key.reqtime.strftime("%d %H:%M"))
    

v=c.value_set.all()
#print('now: ', datetime.now().strftime("%d %H:%M"))
#print('less than: ',t.strftime("%d %H:%M"))
print('all')
for  key in v:
    print(key.id, " ", key.reqtime.strftime("%d %H:%M"))

('now: ', '26 17:45')
('gr than: ', '26 17:40')
(6091, ' ', '26 16:25')
(6194, ' ', '26 16:26')
(6297, ' ', '26 16:27')
(6400, ' ', '26 16:27')
(6503, ' ', '26 16:32')
(6606, ' ', '26 16:39')
(6709, ' ', '26 16:46')
(6812, ' ', '26 16:53')
(6915, ' ', '26 17:00')
(7018, ' ', '26 17:19')
(7121, ' ', '26 17:39')
all
(6606, ' ', '26 16:39')
(6709, ' ', '26 16:46')
(6812, ' ', '26 16:53')
(6915, ' ', '26 17:00')
(7018, ' ', '26 17:19')
(7121, ' ', '26 17:39')
(6091, ' ', '26 16:25')
(6194, ' ', '26 16:26')
(6297, ' ', '26 16:27')
(6400, ' ', '26 16:27')
(6503, ' ', '26 16:32')


In [35]:
c = Coin.objects.get(id=2).order

2018-01-25 19:13:14.760075
2018-01-25 19:20:14.964577
2018-01-25 19:27:14.901505
2018-01-25 19:34:15.070650
2018-01-25 19:41:15.240093
2018-01-26 03:16:01.394752
2018-01-26 03:23:01.383307
2018-01-26 03:30:01.527441
2018-01-26 03:37:01.569885
2018-01-26 03:44:01.648774
2018-01-26 05:19:28.892740
2018-01-26 05:26:28.674522
2018-01-26 05:33:28.836932
2018-01-26 05:40:28.809947
2018-01-26 06:22:43.058629
2018-01-26 06:29:43.317684
2018-01-26 06:36:43.445547
2018-01-26 06:43:43.452467
2018-01-26 06:50:43.519288
2018-01-26 06:57:43.613583
2018-01-26 07:04:43.559746
2018-01-25 19:06:14.752527
